In [20]:
#!pip install rtmlib==0.0.13
#!pip install opencv-python==4.11.0.86
#!pip install librosa==0.11.0
#!pip install Cython==3.0.12
# Microsoft Visual C++ 14.0 or greater is required.
# Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
# The pip version of madmon is outdated and the github repo has many changes recently
#!pip uninstall -y madmom
#!conda install git+https://github.com/CPJKU/madmom.git
#!cd madmom && python setup.py develop --user
#!pip install pyaudio==0.2.14
#!pip install beatnet==1.1.1
#!pip install soundfile==0.13.1
# Install FFmpeg in Windows
# Source: https://phoenixnap.com/kb/ffmpeg-windows
#!pip cache purge
#!pip install moviepy==2.1.1

In [19]:
import os

move_filename = '29a07de'
moves_filepath = os.path.join('outputs', 'moves.json')

In [ ]:
!ffmpeg -i inputs/29a07de.mp4 -ab 160k -ac 2 -ar 44100 -vn processed/29a07de.wav

In [15]:
from BeatNet.BeatNet import BeatNet
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
audio_filepath = os.path.join('processed', f'{move_filename}.wav')
estimator = BeatNet(1, mode='offline', inference_model='DBN', plot=[], thread=False)
beats = estimator.process(audio_filepath)
beats

array([[ 0.  ,  2.  ],
       [ 0.34,  1.  ],
       [ 0.72,  2.  ],
       [ 1.06,  1.  ],
       [ 1.42,  2.  ],
       [ 1.8 ,  1.  ],
       [ 2.14,  2.  ],
       [ 2.48,  1.  ],
       [ 2.82,  2.  ],
       [ 3.16,  1.  ],
       [ 3.5 ,  2.  ],
       [ 3.86,  1.  ],
       [ 4.18,  2.  ],
       [ 4.52,  1.  ],
       [ 4.88,  2.  ],
       [ 5.2 ,  1.  ],
       [ 5.56,  2.  ],
       [ 5.88,  1.  ],
       [ 6.22,  2.  ],
       [ 6.56,  1.  ],
       [ 6.92,  2.  ],
       [ 7.26,  1.  ],
       [ 7.6 ,  2.  ],
       [ 7.94,  1.  ],
       [ 8.28,  2.  ],
       [ 8.62,  1.  ],
       [ 8.96,  2.  ],
       [ 9.3 ,  1.  ],
       [ 9.62,  2.  ],
       [ 9.96,  1.  ],
       [10.3 ,  2.  ],
       [10.66,  1.  ],
       [10.98,  2.  ],
       [11.32,  1.  ],
       [11.66,  2.  ],
       [12.  ,  1.  ],
       [12.36,  2.  ],
       [12.68,  1.  ],
       [13.02,  2.  ],
       [13.36,  1.  ],
       [13.7 ,  2.  ]])

In [17]:
offset = 1

In [98]:
from moviepy import VideoFileClip, TextClip, CompositeVideoClip

video_file = f"inputs/{move_filename}.mp4"
output_file = f"processed/{move_filename}_counts.mp4"


# Load the video
video_clip = VideoFileClip(video_file)

start_counting = False

for index, x in enumerate(beats):
    start_time = x[0]
    end_time = beats[index + 1][0] if index + 1 < len(beats) else x[0]

    start_counting = start_counting or x[1] == 1

    if start_counting:
        # Create the text clip
        text_clip = TextClip(font='arial.ttf', text=f'{(index - offset) % 8 + 1}', font_size=70, color='white', duration=end_time - start_time)
        
        # Set the duration and position of the text clip
        text_clip = text_clip.with_position(('center', 'bottom'))
        
        # Set the start time of the text clip
        text_clip = text_clip.with_start(start_time)
        
        # Composite the text clip on the video
        video_clip = CompositeVideoClip([video_clip, text_clip])

# Write the output video file
video_clip.write_videofile(output_file, codec='libx264')

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'mp42isom'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [480, 480], 'bitrate': 708, 'fps': 30.04, 'codec_name': 'h264', 'profile': '(Baseline)', 'metadata': {'Metadata': '', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 63, 'metadata': {'Metadata': '', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 13.98, 'bitrate': 775, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Baseline)', 'video_size': [480, 480], 'video_bitrate': 708, 'video_fps': 30.04, 'default_audio_input_number': 0, 'default_audio_stream_number': 1, 'audio_fps': 44100, 'audio_bitrate': 63, 'video_duration': 13.98,

MoviePy - Done.
MoviePy - Writing video processed/29a07de_counts.mp4



MoviePy - Done !
MoviePy - video ready processed/29a07de_counts.mp4


In [25]:
import json

# Open and read the JSON file
with open(moves_filepath, 'r') as file:
    data = json.load(file)

# After verifying that the counts are Okay, I can save the counts
data[move_filename] = []
for index, x in enumerate(beats):
    data[move_filename].append({
        'count': (index - offset) % 8 + 1,
        'time': x[0],
        'keypoints': []
    })

with open(moves_filepath, 'w') as fp:
    json.dump(data, fp)

In [114]:
import cv2

start_counting = False
section = 1
for index, x in enumerate(beats):
    count = (index - offset) % 8 + 1
    vidcap = cv2.VideoCapture(video_file)
    vidcap.set(cv2.CAP_PROP_POS_MSEC, int(x[0] * 1000))      # just cue to position
    success, image = vidcap.read()
    if success:
        cv2.imwrite(os.path.join('processed', 'images', f"{move_filename}_section_{section}_count_{count}.jpg"), image)     # save frame as JPEG file
    
    # Release resources
    vidcap.release()
    cv2.destroyAllWindows()

    if count == 8:
        section += 1

In [1]:
import os

for images in os.listdir(os.path.join('processed', 'images')):
    # check if the image ends with png
    if (images.endswith(".jpg")):
        print(images)

29a07de_section_1_count_8.jpg
29a07de_section_2_count_1.jpg
29a07de_section_2_count_2.jpg
29a07de_section_2_count_3.jpg
29a07de_section_2_count_4.jpg
29a07de_section_2_count_5.jpg
29a07de_section_2_count_6.jpg
29a07de_section_2_count_7.jpg
29a07de_section_2_count_8.jpg
29a07de_section_3_count_1.jpg
29a07de_section_3_count_2.jpg
29a07de_section_3_count_3.jpg
29a07de_section_3_count_4.jpg
29a07de_section_3_count_5.jpg
29a07de_section_3_count_6.jpg
29a07de_section_3_count_7.jpg
29a07de_section_3_count_8.jpg
29a07de_section_4_count_1.jpg
29a07de_section_4_count_2.jpg
29a07de_section_4_count_3.jpg
29a07de_section_4_count_4.jpg
29a07de_section_4_count_5.jpg
29a07de_section_4_count_6.jpg
29a07de_section_4_count_7.jpg
29a07de_section_4_count_8.jpg
29a07de_section_5_count_1.jpg
29a07de_section_5_count_2.jpg
29a07de_section_5_count_3.jpg
29a07de_section_5_count_4.jpg
29a07de_section_5_count_5.jpg
29a07de_section_5_count_6.jpg
29a07de_section_5_count_7.jpg
29a07de_section_5_count_8.jpg
29a07de_se